<a href="https://colab.research.google.com/github/dedi-arianto/AUGMENTATION_GAN/blob/master/Ollama_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Ollama in Colab
---

[![5aharsh/collama](https://raw.githubusercontent.com/5aharsh/collama/main/assets/banner.png)](https://github.com/5aharsh/collama)

This is an example notebook which demonstrates how to run Ollama inside a Colab instance. With this you can run pretty much any small to medium sized models offerred by Ollama for free.

For the list of available models check [models being offerred by Ollama](https://ollama.com/library).


## Before you proceed
---

Since by default the runtime type of Colab instance is CPU based, in order to use LLM models make sure to change your runtime type to T4 GPU (or better if you're a paid Colab user). This can be done by going to **Runtime > Change runtime type**.

While running your script be mindful of the resources you're using. This can be tracked at **Runtime > View resources**.

## Running the notebook
---

After configuring the runtime just run it with **Runtime > Run all**. And you can start tinkering around. This example uses [Llama 3.2](https://ollama.com/library/llama3.2) to generate a response from a prompted question using [LangChain Ollama Integration](https://python.langchain.com/docs/integrations/chat/ollama/).

## Installing Dependencies
---

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`




In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,228 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,862 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all P

## Running Ollama
---

In order to use Ollama it needs to run as a service in background parallel to your scripts. Becasue Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.

Service can be started by command `ollama serve`.

`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [2]:
!pip install ollama

In [3]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

## Pulling Model
---

Download the LLM model using `ollama pull llama3.2`.

For other models check https://ollama.com/library

In [4]:
!ollama pull qwen3:4b

In [9]:
from ollama import Client

# Masukkan API Key Anda di sini
API_KEY = '4bc1342bde7d433288db86478e6371ac.PYIQXP3m_Cn-sDQcNBxmaOjl'

# Inisialisasi client dengan header otentikasi
client = Client(headers={'Authorization': f'Bearer {API_KEY}'})

available_tools = {
    'web_search': client.web_search,
    'web_fetch': client.web_fetch
}

messages = [{'role': 'user', 'content': "what are the apparel collections in Q3 2025 does adidas has the most remarkable performance on?, yes right now is 7 jan 2026 and yes there is article in adidas press release that has sentence about apparel"}]

while True:
    response = client.chat(
        model='qwen3:4b',
        messages=messages,
        tools=[client.web_search, client.web_fetch],
        think=True
    )

    if response.message.thinking:
        print('Thinking: ', response.message.thinking)

    if response.message.content:
        print('Content: ', response.message.content)

    messages.append(response.message)

    if response.message.tool_calls:
        print('Tool calls: ', response.message.tool_calls)
        for tool_call in response.message.tool_calls:
            function_to_call = available_tools.get(tool_call.function.name)
            if function_to_call:
                args = tool_call.function.arguments
                # Menjalankan fungsi melalui instance client yang sudah terautentikasi
                result = function_to_call(**args)
                print('Result: ', str(result)[:200] + '...')

                messages.append({
                    'role': 'tool',
                    'content': str(result)[:8000],
                    'tool_name': tool_call.function.name
                })
            else:
                messages.append({
                    'role': 'tool',
                    'content': f'Tool {tool_call.function.name} not found',
                    'tool_name': tool_call.function.name
                })
    else:
        break

Thinking:  Okay, let's tackle this user query. The user is asking about Adidas' apparel collections in Q3 2025 and whether Adidas has the most remarkable performance in that period. They mention that it's currently January 7, 2026, and there's an article in Adidas' press release with a sentence about apparel.

First, I need to check if the user is referring to Q3 2025. Wait, the current date they provided is January 7, 2026. So Q3 2025 would have already passed since Q3 is July to September. That's a bit confusing because if it's January 2026, then Q3 2025 is in the past. Maybe they meant Q3 2026? But the user specifically says Q3 2025. Hmm, but the user also mentions there's a press release article from Adidas that has a sentence about apparel. Since it's now 2026, the press release might be from a previous period.

Wait, the user says "yes there is article in adidas press release that has sentence about apparel". So they believe there's an existing press release that talks about appa

## And that's it!
---

With this you should be able to freely play around with the models in your scripts. Following is an example using `langchain-ollama` to answer a simple prompt.

If you have a use-case that can help out others feel free to add your notebook to [Collama](https://github.com/5aharsh/collama/fork)

In [ ]:
!pip install langchain-ollama

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from IPython.display import Markdown

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = prompt | model

display(Markdown(chain.invoke({"question": "What's the length of hypotenuse in a right angled triangle"})))